# Analysis

In [2]:
import pandas as pd
import numpy as np


In [3]:
results_df = pd.read_csv("results_cv_2.csv")
results_df

,Dataset Name,Path Type,Acceleration Type,Path Direction,Path Number,Sample Rate,Prediction Horizon,Prediction Model,Avg. Lin. Vel.,Average ADE,Average ADE (Normalized),Average FDE,Average FDE (Normalized)
0,VL_C_F_TD_3,VL_C,F,TD,3,full,4,CV,False,10.610816,3.987746,17.300369,1.005540
1,VL_C_F_TD_3,VL_C,F,TD,3,10hz,4,CV,False,22.825323,4.625405,37.339447,0.995166
2,VL_C_F_BU_5,VL_C,F,BU,5,full,4,CV,False,10.443218,2.707653,18.224721,0.961869
3,VL_C_F_BU_5,VL_C,F,BU,5,10hz,4,CV,False,31.477282,4.497487,47.276116,0.922052
4,VL_C_F_BU_6,VL_C,F,BU,6,full,4,CV,False,11.665653,3.470563,18.802438,0.843404
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1025,CP_C_F_CCW_2,CP_C,F,CCW,2,10hz,4,CV,True,22.074105,3.660401,32.766066,0.886578
1026,CP_C_F_CCW_3,CP_C,F,CCW,3,full,4,CV,True,10.018734,2.532712,16.197457,0.819151
1027,CP_C_F_CCW_3,CP_C,F,CCW,3,10hz,4,CV,True,25.323774,3.557619,38.037703,0.824603
1028,CP_C_A_CW_1,CP_C,A,CW,1,full,4,CV,True,10.466486,3.126738,16.736470,0.840685


In [ ]:
results_df_filtered = results_df[
    (results_df['sample_rate'].isin(['10hz'])) & 
    (results_df['ph'] == 4) & 
    (results_df['acceleration_type'] == 'F')
    ]

# Same path type for all straight lines
results_df_filtered.loc[results_df_filtered['path_type'].isin(['VL_C', 'HL_C', 'DL_C']), 'path_type'] = 'Straight'

results_df_filtered.groupby([
    # 'dataset_name',
    'path_type',
    'sample_rate',
    'ph',
    "speed_avg",
    'prediction_model'
                    ]).agg({'average_error': ['mean', 'std'],
    'average_error_normal': ['mean', 'std']}).reset_index()

Others

In [ ]:
results_df = pd.read_csv("results_df_both_110425_1.csv")

results_df_filtered = results_df[
    (results_df['sample_rate'].isin(['2.5hz'])) & 
    (results_df['ph'] == 1) 
    # (results_df['dataset_name'] == 'DL_C_F_BRTL_2')
    # (results_df['acceleration_type'] == 'F')
    # (results_df['speed_avg'] == True) &
    # (results_df['path_type'] == 'IL_C')
    ]

min_error_dataset = results_df_filtered.loc[results_df_filtered['average_error'].idxmin(), 'dataset_name']
print(f"Dataset with minimum average_error: {min_error_dataset}")
min_error_value = results_df_filtered['average_error'].min()
print(f"Minimum average_error value: {min_error_value}")

# results_df_filtered.sort_values(by='dataset_name')
# results_df_filtered
# # results_df_filtered
results_df_filtered.loc[results_df_filtered['path_type'].isin(['VL_C', 'HL_C', 'DL_C']), 'path_type'] = 'Straight'

results_df_filtered.groupby([
    'path_type',
    # 'acceleration_type',
    'sample_rate',
    'ph',
    'prediction_model',
    # 'speed_avg'
                    ]).agg({'average_error': 'mean',
    'average_error_normal': 'mean'}).reset_index()

# # results_df_filtered

In [ ]:
# results_df_filtered = results_df[
#     (results_df['sample_rate'].isin(['2.5hz'])) & 
#     (results_df['ph'] == 1) & 
#     (results_df['dataset_name'] == 'CP_C_F_CCW_2') &
#     (results_df['acceleration_type'] == 'F') &
#     (results_df['speed_avg'] == False) &
#     (results_df['path_type'] == 'CP_C')
#     ]

results_df_filtered = pd.read_csv("results_df_both_110425_1.csv")

# results_df_filtered
results_df_filtered.loc[results_df_filtered['path_type'].isin(['VL_C', 'HL_C', 'DL_C']), 'path_type'] = 'Straight'

agg_df = results_df_filtered.groupby([
    'path_type',
    'acceleration_type',
    'sample_rate',
    'ph',
    'prediction_model',
    'speed_avg'
                    ]).agg({'average_error': 'mean',
    'average_error_normal': 'mean'}).reset_index()

# results_df_filtered

In [ ]:
# Copy the dataframe to avoid modifying the original
df_encoded = agg_df.copy()

# Encode categorical variables using label encoding
categorical_columns = ['acceleration_type', 'sample_rate', 'prediction_model', 'speed_avg', 'path_type']

for col in categorical_columns:
    df_encoded[col] = df_encoded[col].astype('category').cat.codes

# Compute the correlation matrix
correlation_matrix = df_encoded.corr(numeric_only=True)

# Extract only the correlations with the target variables
target_corr = correlation_matrix[['average_error', 'average_error_normal']].drop(['average_error', 'average_error_normal'])

# Display correlation with average_error and average_error_normal
print("Correlation with average_error and average_error_normal:")
print(target_corr.sort_values(by='average_error', ascending=False))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 5))
sns.heatmap(target_corr, annot=True, cmap='coolwarm')
plt.title("Correlation of Parameters with Error Metrics")
plt.show()


In [ ]:
results_df_filtered = pd.read_csv("results_df_both_110425_1.csv")

results_df_filtered_1 = results_df[
    (results_df['sample_rate'].isin(['10hz'])) &
    (results_df['ph'] == 4)
    # (results_df['dataset_name'] == 'CP_C_F_CCW_2') &
    # (results_df['acceleration_type'] == 'F') &
    # (results_df['speed_avg'] == False) &
    # (results_df['path_type'] == 'CP_C')
    ]

results_df_filtered_2 = results_df[
    (results_df['sample_rate'].isin(['2.5hz'])) &
    (results_df['ph'] == 1)
    # (results_df['dataset_name'] == 'CP_C_F_CCW_2') &
    # (results_df['acceleration_type'] == 'F') &
    # (results_df['speed_avg'] == False) &
    # (results_df['path_type'] == 'CP_C')
    ]

results_df_filtered = pd.concat([results_df_filtered_1, results_df_filtered_2]).drop_duplicates(keep='last').reset_index(drop=True)

# Copy and encode again if you didn't keep df_encoded
df_encoded = results_df_filtered.copy()

# Convert categorical columns to numeric codes
categorical_columns = ['acceleration_type', 'sample_rate', 'path_type', 'speed_avg']

for col in categorical_columns:
    df_encoded[col] = df_encoded[col].astype('category').cat.codes

# Also encode 'prediction_model' to filter more easily
df_encoded['prediction_model'] = df_encoded['prediction_model'].astype(str)

# Filter for each model
ctrv_df = df_encoded[df_encoded['prediction_model'] == 'CTRV']
cv_df = df_encoded[df_encoded['prediction_model'] == 'CV']

# List of parameters to correlate
features = ['acceleration_type', 'sample_rate', 'ph', 'speed_avg', 'path_type']

# Compute correlations
ctrv_corr = ctrv_df[features + ['average_error']].corr().loc[features, 'average_error']
cv_corr = cv_df[features + ['average_error']].corr().loc[features, 'average_error']

# Create a comparison DataFrame
comparison_df = pd.DataFrame({
    'CTRV': ctrv_corr,
    'CV': cv_corr
}).sort_values(by='CTRV', ascending=False)

print("Parameter correlation with average_error (CTRV vs CV):")
print(comparison_df)


In [ ]:
comparison_df.plot(kind='bar', figsize=(10, 5))
plt.title('Comparison of Parameter Correlation with average_error (CTRV vs CV)')
plt.ylabel('Correlation Coefficient')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()
